In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")


indices<-Indices()

In [ ]:
nifty_50 <- indices$NseTimeSeries() %>%
                 filter(NAME == "NIFTY 50") %>%  
                 arrange(desc(TIME_STAMP)) %>%
                 print(n=4)

In [ ]:
nifty <- as.data.frame(nifty_50)

In [ ]:
abs_returns <- (c(diff(nifty$CLOSE,lag=1)*(-1),0)) 

nifty <- data.frame(nifty, abs_returns)

n<-1
per_returns=rep(0,7055)
while(n<=(length(nifty$abs_returns)-1)){
    per_returns[n] = ((nifty$abs_returns[n])/(nifty$CLOSE[n+1]))*100
    n = n+1
}

length(per_returns)

In [ ]:
length(abs_returns)

In [ ]:
head(nifty)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)